In [5]:
from alpaca_sdk import *
import seaborn as sns

today = dt.now(tz=ny_eastern).date()
raw_bngo = stock_data('RCAT', today, 10)


In [6]:
print(raw_bngo.iloc[-1])

timestamp      2025-01-24 05:00:00+00:00
open                                8.41
high                                9.83
low                                 8.29
close                               8.63
volume                        12988375.0
trade_count                      77702.0
vwap                            9.123961
smaLow                          9.102857
smaHigh                        11.056667
so                              7.258819
atr                             1.660944
rsi                            44.618422
Name: 938, dtype: object


In [7]:
money = 100
n = 0
buy_signal = False
assets = []
number = []
total = []
money_hist = []

for i in range(len(raw_bngo)):
    logic_buy = raw_bngo['smaLow'].iloc[i] > raw_bngo['smaHigh'].iloc[i] and raw_bngo['rsi'].iloc[i] < 40 and raw_bngo['so'].iloc[i] < 30
    logic_sell = raw_bngo['smaLow'].iloc[i] < raw_bngo['smaHigh'].iloc[i] and raw_bngo['rsi'].iloc[i] > 60  and raw_bngo['so'].iloc[i] > 70
    
    if logic_buy and not buy_signal:
        buy_signal = True
        n = int(money / raw_bngo['close'].iloc[i])
        asset = raw_bngo['close'].iloc[i] * n
        money -= asset
        
    elif logic_sell and buy_signal:
        buy_signal = False
        money += raw_bngo['close'].iloc[i] * n
        raw_bngo['total'] = money + asset
        asset = 0
        n = 0

    money_hist.append(money)
    assets.append(raw_bngo['close'].iloc[i] * n)
    number.append(n)
    total.append(raw_bngo['close'].iloc[i] * n + money)

raw_bngo['money'] = money_hist
raw_bngo['asset'] = assets
raw_bngo['number'] = number
raw_bngo['total'] = total

In [8]:
raw_bngo

,timestamp,open,high,low,close,volume,trade_count,vwap,smaLow,smaHigh,so,atr,rsi,total,money,asset,number
0,2021-04-30 04:00:00+00:00,4.2625,4.5500,4.00,4.040,942325.0,5750.0,4.261457,NaN,NaN,NaN,0.000000,NaN,100.00,100.00,0.0,0
1,2021-05-03 04:00:00+00:00,4.1394,4.1498,3.70,3.930,348170.0,2233.0,3.835043,NaN,NaN,NaN,0.000000,NaN,100.00,100.00,0.0,0
2,2021-05-04 04:00:00+00:00,3.8000,3.8800,3.66,3.840,564998.0,3740.0,3.764966,NaN,NaN,NaN,0.000000,NaN,100.00,100.00,0.0,0
3,2021-05-05 04:00:00+00:00,3.8000,4.0500,3.65,3.840,323491.0,2258.0,3.807519,NaN,NaN,NaN,0.000000,NaN,100.00,100.00,0.0,0
4,2021-05-06 04:00:00+00:00,3.9300,4.0600,3.78,3.870,259598.0,1551.0,3.889986,NaN,NaN,NaN,0.000000,NaN,100.00,100.00,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,2025-01-17 05:00:00+00:00,10.1000,10.1000,8.29,8.555,13262663.0,86262.0,9.105186,10.018571,11.223571,3.794550,1.823586,43.702851,513.64,513.64,0.0,0
935,2025-01-21 05:00:00+00:00,8.5600,8.8900,8.11,8.460,9411524.0,67514.0,8.469310,9.537143,11.165952,4.885743,1.749044,43.398999,513.64,513.64,0.0,0
936,2025-01-22 05:00:00+00:00,8.9150,9.4400,8.47,8.750,12814711.0,68729.0,8.813505,9.354286,11.194524,8.933931,1.694112,44.663793,513.64,513.64,0.0,0
937,2025-01-23 05:00:00+00:00,8.7100,9.6100,8.25,8.310,9685232.0,67174.0,8.665809,9.208571,11.157619,2.791853,1.670247,43.090476,513.64,513.64,0.0,0


In [9]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=raw_bngo['timestamp'], y=raw_bngo['close'], mode='lines', name='Close'))
fig.add_trace(go.Scatter(x=raw_bngo['timestamp'], y=raw_bngo['smaLow'], mode='lines', name='sma low'))
fig.add_trace(go.Scatter(x=raw_bngo['timestamp'], y=raw_bngo['smaHigh'], mode='lines', name='sma high'))
fig.show()


In [10]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=raw_bngo['timestamp'], y=raw_bngo['total'], mode='lines', name='Close'))
fig.show()

In [11]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=raw_bngo['timestamp'], y=raw_bngo['number'], mode='lines', name='Close'))
# fig.add_trace(go.Scatter(x=raw_bngo['timestamp'], y=raw_bngo['number'], mode='lines', name='n'))
fig.show()
